## Structural Equation Test to the Bell DAG:

given Bell scenario correlations
$P(A,B,X,Y) =  \sum_\lambda P(A| X, \lambda) P(B|Y, \lambda) P(X)P(Y)P(\lambda) \qquad \forall \lambda \in [0,card]$  

In [1]:
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

In [2]:
card_A = 2
card_B = 2
card_X = 2
card_Y = 2

In [3]:
P_AB_giv_XY_dict = {(0, 0, 0, 0): 1/2,
    (0, 0, 0, 1): 1/4,
    (0, 0, 1, 0): 1/4,
    (0, 0, 1, 1): 1/2,
    (0, 1, 0, 0): 0,
    (0, 1, 0, 1): 1/4,
    (0, 1, 1, 0): 1/4,
    (0, 1, 1, 1): 0,
    (1, 0, 0, 0): 0,
    (1, 0, 0, 1): 1/4,
    (1, 0, 1, 0): 1/4,
    (1, 0, 1, 1): 0,
    (1, 1, 0, 0): 1/2,
    (1, 1, 0, 1): 1/4,
    (1, 1, 1, 0): 1/4,
    (1, 1, 1, 1): 1/2}
    
dist_ABXY = P_AB_giv_XY_dict

In [11]:
dist_ABXY = {}
sum(dist_ABXY.values())


# # get P_AB|XY from P_ABXY
def P_AB_giv_XY(A,B, X, Y):
    P_ABXY = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B and x==X and y==Y])
    P_AB = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B])
    return P_ABXY/P_AB

In [4]:
# # distribution feasible?
# sum(dist_ABXY.values()) == 1

# # get P_AB|XY from P_ABXY
# def P_AB_giv_XY(A,B, X, Y):
#     P_ABXY = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B and x==X and y==Y])
#     P_AB = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B])
#     return P_ABXY/P_AB


# independences
# def P_XY(x,y):
#     return sum([dist_ABXY[(a,b,x,y)] for a,b in itertools.product(range(card_A), range(card_B))])
# P(XY) = P(X)P(Y)
# for x,y in np.ndindex(card_X, card_Y):
#     print(P_XY(x,y) == P_X(x) * P_Y(y))

In [6]:
## first check if DAG is compatible with distributions, then if compatible, find minimal card_l

#get is_compatible variable value from deterministic_bell_compatibility.py
# from is_dist_bell_compatible import main

# if not(is_compatible):
#     raise SystemExit("distribution not compatible w/ Bell DAG!")

In [5]:
m = gp.Model()
card_l = 0 # hidden common cause
max_tries = 0

while True:
    m.reset()
    m.params.NonConvex = 2  # Using quadratic equality constraints.
    card_l += 1
    max_tries += 1

    # variables
    P_l = m.addMVar(card_l, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="P_l")
    P_A_giv_X_l = m.addMVar(shape = (card_A, card_X, card_l), vtype=GRB.CONTINUOUS, name="P(A|X,l)", lb=0, ub=1)
    P_B_giv_Y_l = m.addMVar(shape = (card_B, card_Y, card_l), vtype=GRB.CONTINUOUS, name="P(B|Y,l)", lb=0, ub=1)
    prod = m.addMVar(shape = (card_A, card_B, card_X, card_Y, card_l), vtype=GRB.CONTINUOUS, name="quad_prod", lb=0, ub=1) # P(A|X,l) * P(B|Y,l)
    m.update()

    def P_Y(Y):
        return sum([dist_ABXY[(a,b,x,Y)] for a,b,x in np.ndindex(card_A, card_B, card_X)])
    def P_X(X):
        return sum([dist_ABXY[(a,b,X,y)] for a,b,y in np.ndindex(card_A, card_B, card_Y)])
    def P_XYl(x,y,l):
        return P_X(x) * P_Y(y) * P_l[l]

    for a,b,x,y, l in np.ndindex(card_A, card_B, card_X, card_Y, card_l):
        m.addConstr(prod[a,b,x,y,l] == P_A_giv_X_l[a,x,l] * P_B_giv_Y_l[b,y,l])

    # structural eqn
    for a,b,x,y in np.ndindex(card_A, card_B, card_X, card_Y):
        dist_ABXY[a,b,x,y] == sum([prod[a,b,x,y,l] * P_XYl(x,y,l) for l in range(card_l)])

    # sums to 1
    m.addConstr(sum([P_l[l] for l in range(card_l)]) == 1, "sum_P_l = 1")
    m.addConstr(P_A_giv_X_l[0,:,:]+P_A_giv_X_l[1,:,:] == 1 , "sum_P_A_giv_X_l = 1")
    m.addConstr(P_B_giv_Y_l[0,:,:]+P_B_giv_Y_l[1,:,:] == 1 , "sum_P_B_giv_Y_l = 1")

    if m.optimize() == GRB.OPTIMAL:
        print('Optimal solution found with card_l =', card_l)
        break

    print('Optimal solution not found with card_l =', card_l)
    if max_tries >= 10:
        print('Loop interupted, max tries reached at card_l =', card_l)
        break

Discarded solution information
Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 25 columns and 9 nonzeros
Model fingerprint: 0x619ea215
Model has 16 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 1 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 68 rows, 24 columns, 136 nonzeros
Presolved model has 16 bilinear constraint(s)
Variable types: 24 continuous, 0 integer (0 binary)
Found heuristic solution: object

In [ ]:
%%capture
m.optimize()